In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sys, io, os, errno, fileinput, csv
import collections as cl
from os import path
from sklearn.pipeline import Pipeline
import matplotlib as plt
import seaborn as sb
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow import keras as kr
import itertools
import warnings
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer



trainset_file = "../../Datasets/the_onion.csv"
train_df = pd.read_csv(trainset_file,  sep=',')

train_df

,Unnamed: 0,author,domain,num_comments,score,subreddit,timestamp,title
0,0,DisastrousCandy3,theonion.com,0,1,TheOnion,1555122997,Rising Income Inequality Causing Wealthy Ameri...
1,1,ZVan12,kwqc.com,1,1,TheOnion,1555118645,Woman Crashes Car After Seeing Spider Riding S...
2,2,MyNameIsGriffon,theonion.com,0,1,TheOnion,1555106860,Lockheed Martin Executive Fondly Recalls Humbl...
3,3,MyNameIsGriffon,theonion.com,0,1,TheOnion,1555106859,Experts Warn Prosecuting Assange Creates Slipp...
4,4,MyNameIsGriffon,sports.theonion.com,0,1,TheOnion,1555106858,Kid About To Meet Brooklyn Nets Must Not Be Ve...
5,5,SlovenianCat,youtu.be,0,1,TheOnion,1555106482,Lawmaker's War Hero Son Would Have Wanted Road...
6,6,aresef,entertainment.theonion.com,0,1,TheOnion,1555105378,Nation Admits They Only Care About Freedom Of ...
7,7,MyNameIsGriffon,theonion.com,1,1,TheOnion,1555099714,Media Condemns Julian Assange For Reckless Exp...
8,8,MyNameIsGriffon,sports.theonion.com,0,1,TheOnion,1555099714,Duke Anthropology Professor Devastated To Lear...
9,9,Bearsrock2468,i.redd.it,2,1,TheOnion,1555095395,I know it's satire but


In [ ]:
title_vectorizer = TfidfVectorizer(max_features=8192, stop_words='english', lowercase=True, ngram_range=(1, 3),smooth_idf=False)
auth_vectorizer = TfidfVectorizer(max_features=8192, stop_words='english', lowercase=True, ngram_range=(1, 3),smooth_idf=False)
url_vectorizer = TfidfVectorizer(max_features=8192, stop_words='english', lowercase=True, ngram_range=(1, 3),smooth_idf=False)

title_vectorizer.fit(train_df['title'].astype('U'))
title_vectorizer.transform(train_df['title'].astype('U'))
auth_vectorizer.fit(train_df['author'].astype('U'))
auth_vectorizer.transform(train_df['author'].astype('U'))
url_vectorizer.fit(train_df['domain'].astype('U'))
url_vectorizer.transform(train_df['domain'].astype('U'))

with open('../../models/onion_title_tfidf.pickle', 'wb') as fin:
    pickle.dump(title_vectorizer, fin)
    
with open('../../models/onion_author_tfidf.pickle', 'wb') as fin:
    pickle.dump(auth_vectorizer, fin)
    
with open('../../models/onion_domain_tfidf.pickle', 'wb') as fin:
    pickle.dump(url_vectorizer, fin)